<a href="https://colab.research.google.com/github/shachi-i/Volatility-forecasting/blob/main/lstm_garch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
from pandas_datareader import data
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

%matplotlib inline

from tensorflow import keras
from tensorflow.keras import layers


import datetime



from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score





In [2]:




from keras.models import Model
from keras.layers import *



import tensorflow as tf

In [3]:
import yfinance as yf

tickers = ["^N225", "^GDAXI"]  # Nikkei 225 (Japan) + DAX 30 (Germany)
data = yf.download(tickers, start="2010-01-01", end="2025-01-01")['Close']


/tmp/ipython-input-643132691.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2010-01-01", end="2025-01-01")['Close']
[*********************100%***********************]  2 of 2 completed


In [4]:
import numpy as np
returns = np.log(data / data.shift(1)).dropna()
realized_vol = returns.rolling(window=21).std()  # 1-month rolling vol


bench mark statistical model

In [5]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 13.4 MB/s eta 0:00:00


Statistical forecasting for N225 (JP)

In [6]:
from arch import arch_model

garch = arch_model(returns['^N225'], vol='Garch', p=1, q=1)
res = garch.fit()
preds = res.forecast(horizon=1, reindex=False)


Iteration:      1,   Func. Count:      6,   Neg. LLF: 9.249693234114344e+20
Iteration:      2,   Func. Count:     19,   Neg. LLF: 2.772335523364103e+19
Optimization terminated successfully    (Exit mode 0)
            Current function value: -10098.748206934717
            Iterations: 2
            Function evaluations: 29
            Gradient evaluations: 2


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001755. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


In [7]:
egarch11 = arch_model(returns['^N225'], vol='EGarch', p=1, q=1, dist='normal')
res_egarch = egarch11.fit(update_freq=5, disp='off')


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001755. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


generate volatility forecasts

In [8]:
forecast_garch = res.forecast(horizon=1, reindex=False)
forecast_egarch = res_egarch.forecast(horizon=1, reindex=False)


vol_garch = np.sqrt(forecast_garch.variance.values[-1, :])[0]
vol_egarch = np.sqrt(forecast_egarch.variance.values[-1, :])[0]



In [9]:
realized = returns[-len(returns):] ** 2
predicted = vol_garch ** 2

In [10]:
rmse = np.sqrt(np.mean((predicted - realized) ** 2))


In [11]:
mae = np.mean(np.abs(predicted - realized))


In [12]:
qlike = np.mean(np.log(predicted) + (realized / predicted))


In [13]:
e_realized = returns[-len(returns):] ** 2
e_predicted = vol_egarch ** 2
rmse_egarch = np.sqrt(np.mean((e_predicted - e_realized) ** 2))
mae_egarch = np.mean(np.abs(e_predicted - e_realized))
qlike_egarch = np.mean(np.log(e_predicted) + (e_realized / e_predicted))

In [14]:
import pandas as pd

comparison = pd.DataFrame(
    [[rmse, mae, qlike]],
    index=["GARCH(1,1)","EGARCH(1,1)"],
    columns=["RMSE", "MAE", "QLIKE"]
)
print(comparison)



                 RMSE       MAE     QLIKE
GARCH(1,1)   0.000533  0.000159 -7.435138
EGARCH(1,1)  0.000533  0.000159 -7.435138


Statistical forecasting for DAX 30 (DE)

In [15]:
from arch import arch_model

garch1 = arch_model(returns['^GDAXI'], vol='Garch', p=1, q=1)
res1 = garch.fit()
preds1 = res.forecast(horizon=1, reindex=False)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 9.249693234114344e+20
Iteration:      2,   Func. Count:     19,   Neg. LLF: 2.772335523364103e+19
Optimization terminated successfully    (Exit mode 0)
            Current function value: -10098.748206934717
            Iterations: 2
            Function evaluations: 29
            Gradient evaluations: 2


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001755. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


In [16]:
egarch11_1 = arch_model(returns['^GDAXI'], vol='EGarch', p=1, q=1, dist='normal')
res_egarch_1 = egarch11_1.fit(update_freq=5, disp='off')

/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001521. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


In [17]:
forecast_garch_1 = res1.forecast(horizon=1, reindex=False)
forecast_egarch_1 = res_egarch_1.forecast(horizon=1, reindex=False)


vol_garch1 = np.sqrt(forecast_garch_1.variance.values[-1, :])[0]
vol_egarch1 = np.sqrt(forecast_egarch_1.variance.values[-1, :])[0]


In [18]:
realized1 = returns[-len(returns):] ** 2
predicted1 = vol_garch1 ** 2
rmse1 = np.sqrt(np.mean((predicted1 - realized1) ** 2))
mae1 = np.mean(np.abs(predicted1 - realized1))
qlike1 = np.mean(np.log(predicted1) + (realized1 / predicted1))
e_realized1 = returns[-len(returns):] ** 2
e_predicted1 = vol_egarch1 ** 2
rmse_egarch1 = np.sqrt(np.mean((e_predicted1 - e_realized1) ** 2))
mae_egarch1 = np.mean(np.abs(e_predicted1 - e_realized1))
qlike_egarch1 = np.mean(np.log(e_predicted1) + (e_realized1 / e_predicted1))

In [19]:
import pandas as pd

comparison = pd.DataFrame(
    [[rmse, mae, qlike]],
    index=["GARCH(1,1)","EGARCH(1,1)"],
    columns=["RMSE", "MAE", "QLIKE"]
)
print(comparison)


                 RMSE       MAE     QLIKE
GARCH(1,1)   0.000533  0.000159 -7.435138
EGARCH(1,1)  0.000533  0.000159 -7.435138


splitting data into train and test sets

Linear Regression

LSTM

In [23]:
import numpy as np

def make_sequences(df, feature_cols, seq_len=21):
    X, y = [], []
    for i in range(seq_len, len(df)-1):
        window = df.iloc[i-seq_len:i][feature_cols].values
        X.append(window)
        y.append(df.iloc[i+1]['rv'])   # next-day realized variance
    return np.array(X), np.array(y)


In [24]:
import numpy as np
import pandas as pd

np.random.seed(42)

# Create 10 years of synthetic data (approx 2500 days)
n = 2500
dates = pd.date_range(start="2010-01-01", periods=n, freq="B")

# Synthetic price path using geometric brownian motion
mu = 0.0002
sigma = 0.01
returns = np.random.normal(mu, sigma, n)
price = 100 * (1 + returns).cumprod()

df_price = pd.DataFrame({
    'Date': dates,
    '^N225': price
}).set_index('Date')


In [25]:
feat = df_price.copy()

feat['^N225_r'] = feat['^N225'].pct_change()
feat['^N225_r_lag_1'] = feat['^N225_r'].shift(1)
feat['^N225_r_lag_2'] = feat['^N225_r'].shift(2)

feat['rv'] = feat['^N225_r']**2
feat['rv_21'] = feat['rv'].rolling(21).sum()

# Fake GARCH variance (for now use rolling variance)
feat['garch_var'] = feat['^N225_r'].rolling(21).var()

feat = feat.dropna()


In [26]:
feature_cols = [
    '^N225_r',
    '^N225_r_lag_1',
    '^N225_r_lag_2',
    'rv_21',
    'garch_var'
]


In [27]:
X, y = make_sequences(feat, feature_cols, seq_len=21)

split = int(0.8 * len(X))

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [28]:

from sklearn.preprocessing import StandardScaler

n_features = X_train.shape[2]

scaler = StandardScaler().fit(X_train.reshape(-1, n_features))

X_train_s = scaler.transform(X_train.reshape(-1, n_features)).reshape(X_train.shape)
X_test_s  = scaler.transform(X_test.reshape(-1, n_features)).reshape(X_test.shape)


In [29]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(21, n_features)),
    layers.GRU(64, return_sequences=False),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        13,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,745 (61.50 KB)

 Trainable params: 15,745 (61.50 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
history = model.fit(
    X_train_s, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0047 - val_loss: 2.8771e-04
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 2.1596e-04 - val_loss: 1.4293e-04
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 1.2601e-04 - val_loss: 8.5493e-05
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 7.3222e-05 - val_loss: 6.3146e-05
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 7.0348e-05 - val_loss: 6.3945e-05
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 5.7819e-05 - val_loss: 4.6346e-05
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 3.8127e-05 - val_loss: 4.7981e-05
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 3.4806e-05 - val_loss: 3.5668e-05
Epoch 9/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 2.7258e-05 - val_loss: 5.1393e-05
Epoch 10/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 2.7176e-05 - val_loss: 4.2765e-05
Epoch 11/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 2.3887e-05 - 

In [31]:
pred_test = model.predict(X_test_s).flatten()


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


In [32]:
def compute_metrics(pred, real):
    rmse = np.sqrt(np.mean((pred - real)**2))
    mae  = np.mean(np.abs(pred - real))
    qlike = np.mean(np.log(pred) + real / pred)
    return rmse, mae, qlike

rmse, mae, qlike = compute_metrics(pred_test, y_test)

print("RMSE:", rmse)
print("MAE:", mae)
print("QLIKE:", qlike)


RMSE: 0.003986820088728256
MAE: 0.0029224991983766565
QLIKE: nan


/tmp/ipython-input-3911984173.py:4: RuntimeWarning: invalid value encountered in log
  qlike = np.mean(np.log(pred) + real / pred)


Hybrid garch lstm

In [33]:
from arch import arch_model

returns = feat['^N225_r']

garch = arch_model(returns, vol='Garch', p=1, q=1).fit(disp='off')
garch_f = garch.forecast(horizon=1, reindex=False)

feat['garch_var'] = garch_f.variance.iloc[:,0]
feat['garch_resid'] = feat['rv'] - feat['garch_var']


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 9.655e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


In [34]:
feature_cols = ['^N225_r','^N225_r_lag_1','^N225_r_lag_2','rv_21','garch_var']


X, y = make_sequences(feat, feature_cols, seq_len=21)
